In [1]:
#!python3 -m pip install --force-reinstall sql_sidekick-0.1.7-py3-none-any.whl

In [2]:
import gc
import json
import logging
import os
from pathlib import Path
from typing import List, Optional

import openai
import toml
import torch

In [3]:
from sidekick.prompter import (data_preview, db_setup, ask,
                               recommend_suggestions)
from sidekick.schema_generator import generate_schema
from sidekick.query import SQLGenerator
from sidekick.utils import (MODEL_CHOICE_MAP_DEFAULT,
                            MODEL_CHOICE_MAP_EVAL_MODE, TASK_CHOICE,
                            get_table_keys, save_query, setup_dir,
                            update_tables)

In [4]:
base_path = "./"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [5]:
# Setup the DB for a single table (WIP to extend to multiple tables)

HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
data_path = "/home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_2"

r, table_info_path = generate_schema(data_path, f"{cache_path}/{table_name}_table_info.jsonl")

llm_response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path
            )

 Information supplied:
 querydb, localhost, sqlite, abc, 5432


2023-11-29 03:32:59.065 | DEBUG    | sidekick.db_config:__init__:42 - Creating SQLite DB: sqlite:///.//db/sqlite/querydb.db
2023-11-29 03:32:59.074 | DEBUG    | sidekick.db_config:create_db:79 - SQLite DB is created when 'engine.connect()' is called


Database created successfully!
Table name: sleep_health_2


2023-11-29 03:32:59.080 | DEBUG    | sidekick.db_config:_extract_schema_info:100 - Schema path: .//var/lib/tmp/sleep_health_2_table_info.jsonl
2023-11-29 03:32:59.084 | DEBUG    | sidekick.db_config:create_table:145 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE


Checked table sleep_health_2 exists in the DB.


2023-11-29 03:32:59.101 | DEBUG    | sidekick.db_config:add_samples:179 - Adding sample values to table: /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv
2023-11-29 03:32:59.107 | DEBUG    | sidekick.db_config:add_samples:185 - Inserting chunk: 0
2023-11-29 03:32:59.210 | INFO     | sidekick.db_config:add_samples:190 - Data inserted into table: sleep_health_2
2023-11-29 03:32:59.218 | INFO     | sidekick.db_config:add_samples:195 - Number of rows inserted: 374


Created a Database querydb. Inserted sample values from /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv into table sleep_health_2, please ask questions!
